## แบบฝึกหัดที่ 8

จากแบบฝึกหัดที่ 7 ให้นิสิตแสดงการ optimize 'n_neighbors' โดยมีรายละเอียดดังนี้  
1. ทำการแบ่ง train และ test set โดยใช้ train_test_split() ซึ่งมี argument test_size=0.3 และ random_state=1234
2. ทำการ optimize 'n_neighbors' โดยใช้ train set ทดสอบค่า 'n_neighbors' ในช่วง 1-30 และประสิทธิภาพที่ใช้วัดมาจาก Stratified 5-fold cross-validation (random_state=1234 ไม่มีการ repeat) โดยพิจารณาจาก auc score
3. สร้างโมเดล KNN จาก train set โดยใช้ 'n_neighbors' ที่ได้จากการ optimize นำโมเดลนี้มาทำนาย test set ในข้อ 1 และวัดประสิทธิภาพของโมเดลโดยแสดง confusion matrix แบบ heatmap และแสดงค่า accuracy, precision, recall, f1, auc

ข้อมูล "Wholesale_customers_data.csv" มีรายละเอียดดังนี้:

**Attribute Information:**  
  
1) Fresh: annual spending (m.u.) on fresh products (Continuous);  
2) Milk: annual spending (m.u.) on milk products (Continuous);  
3) Grocery: annual spending (m.u.)on grocery products (Continuous);  
4) Frozen: annual spending (m.u.)on frozen products (Continuous)  
5) Detergents_paper: annual spending (m.u.) on detergents and paper products (Continuous)  
6) Delicassen: annual spending (m.u.)on and delicatessen products (Continuous);  
7) Channel<sup>*</sup>: customers channel - 1:Horeca (Hotel/Restaurant/Cafe) or 2:Retail channel (Nominal)  
8) Region: customers region - Lisnon, Oporto or Other (Nominal)

<sup>*</sup>Outcome/label variable  
monetary units = m.u.

โมเดลที่สร้างมีเป้าหมายเพื่อการทำนายตัวแปร Channel เช่นเดียวกับแบบฝึกหัดที่ 7

&nbsp;   
#### <font color="#1f7d02">*Read Dataset*</span>

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict

In [20]:
df = pd.read_csv("https://raw.githubusercontent.com/ThammakornS/ProgStat/main/dataset/Wholesale_customers_data.csv")
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [21]:
df.Channel.value_counts()

Channel
1    298
2    142
Name: count, dtype: int64

In [22]:
df = df.astype({
    'Region': 'category',
})
categorical_cols = df.select_dtypes(include=['category']).columns
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)

&nbsp;  
#### <font color="#1f7d02">*Create feature and label data*</span>

In [23]:
X = df.loc[:,~(df.columns=='Channel')]
X

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Region_2,Region_3
0,12669,9656,7561,214,2674,1338,0,1
1,7057,9810,9568,1762,3293,1776,0,1
2,6353,8808,7684,2405,3516,7844,0,1
3,13265,1196,4221,6404,507,1788,0,1
4,22615,5410,7198,3915,1777,5185,0,1
...,...,...,...,...,...,...,...,...
435,29703,12051,16027,13135,182,2204,0,1
436,39228,1431,764,4510,93,2346,0,1
437,14531,15488,30243,437,14841,1867,0,1
438,10290,1981,2232,1038,168,2125,0,1


In [24]:
y = df['Channel']
y

0      2
1      2
2      2
3      1
4      2
      ..
435    1
436    1
437    2
438    1
439    1
Name: Channel, Length: 440, dtype: int64

In [25]:
y.value_counts()

Channel
1    298
2    142
Name: count, dtype: int64

&nbsp;  
#### <font color="#1f7d02">*Train Test Split*</span>

In [26]:
# Your code (use random_state=1234)
# Your code (use random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=1234)

In [27]:
X_train.shape

(308, 8)

In [28]:
X_test.shape

(132, 8)

In [29]:
y_train.value_counts()

Channel
1    211
2     97
Name: count, dtype: int64

In [30]:
y_test.value_counts()

Channel
1    87
2    45
Name: count, dtype: int64

&nbsp;  
#### <font color="#1f7d02">*Feature Data Scaling*</span>

In [31]:
# Your code
df.std(numeric_only=True)

Channel                 0.468052
Fresh               12647.328865
Milk                 7380.377175
Grocery              9503.162829
Frozen               4854.673333
Detergents_Paper     4767.854448
Delicassen           2820.105937
Region_2                0.309234
Region_3                0.450397
dtype: float64

In [35]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
scaler.fit(X_train) #Scaling parameters must based on training data.

X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)
X_train_sc

array([[0.389931  , 0.0644173 , 0.0789312 , ..., 0.02252816, 0.        ,
        1.        ],
       [0.04012645, 0.08304399, 0.14996174, ..., 0.05801001, 0.        ,
        0.        ],
       [0.35844899, 0.01898071, 0.02095347, ..., 0.02774301, 0.        ,
        1.        ],
       ...,
       [0.27900937, 0.02666013, 0.07056706, ..., 0.03861076, 1.        ,
        0.        ],
       [0.15893171, 0.38493798, 0.42781077, ..., 0.05980392, 0.        ,
        0.        ],
       [0.03405305, 0.04946693, 0.149024  , ..., 0.00116813, 1.        ,
        0.        ]])

&nbsp;  
#### <font color="#1f7d02">*Evaluated KNN model on train set when n_neighbors = 1-30 by using stratified 5-fold cv*</span>

In [15]:
# Your code
kf = KFold(n_splits=5,
           shuffle=True,
           random_state=1234)

&nbsp;  
#### <font color="#1f7d02">*Plot performance scores for each 'n_neighbors'*</span>

In [16]:
# Your code

&nbsp;  
#### <font color="#1f7d02">*Use KNN model with optimized 'n_neighbors' to predict test set*</span>

In [17]:
# Your code